In [2]:
import numpy as np
import pandas as pd
import json

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
meta = pd.read_csv("./fishingfinal.csv")
meta = meta.rename(columns={'id':'fishingId'})
meta.head()


,fishingId,longitude,latitude,name,pointName,material,dpwt,address,category,obsCode,obsPostId
0,1,124.729472,37.831778,대청도·소청도,선진포항 앞바다,가는 모래,13~20,인천광역시 옹진군,0,IE_0062,웅진소청초
1,2,124.780222,37.777278,대청도·소청도,곽암 부근,"자갈, 조개껍질",23~29,인천광역시 옹진군,0,IE_0062,웅진소청초
2,3,124.733750,37.751500,대청도·소청도,가두암 부근,가는 모래,26~30,인천광역시 옹진군,0,IE_0062,웅진소청초
3,4,124.664083,37.801278,대청도·소청도,갑죽도 부근,"모래, 조개껍질",22~27,인천광역시 옹진군,0,IE_0062,웅진소청초
4,5,126.317972,37.522833,장봉도,비가지(날가지도) 부근,"모래, 펄",6~11,인천광역시 옹진군,0,IE_0062,웅진소청초


In [29]:
ratings = pd.read_csv("./reviewdata.csv")


ratings = ratings[['id', 'userId', 'fishingId', 'rating']]
ratings = ratings.rename(columns={'id':'reviewId'})

ratings.head()

,reviewId,userId,fishingId,rating
0,1,1838,611,4
1,2,1119,249,3
2,3,770,93,4
3,4,933,618,1
4,5,422,472,4


In [30]:
data = pd.merge(ratings, meta, on='fishingId', how='inner') 
data.head()

,reviewId,userId,fishingId,rating,longitude,latitude,name,pointName,material,dpwt,address,category,obsCode,obsPostId
0,1,1838,611,4,128.216722,34.808194,사량도,하사량도 은포마을 갯바위,펄,5.9~6.3,경상남도 통영시,1,DT_0014,통영
1,2189,255,611,1,128.216722,34.808194,사량도,하사량도 은포마을 갯바위,펄,5.9~6.3,경상남도 통영시,1,DT_0014,통영
2,5345,330,611,4,128.216722,34.808194,사량도,하사량도 은포마을 갯바위,펄,5.9~6.3,경상남도 통영시,1,DT_0014,통영
3,9338,1213,611,2,128.216722,34.808194,사량도,하사량도 은포마을 갯바위,펄,5.9~6.3,경상남도 통영시,1,DT_0014,통영
4,2,1119,249,3,126.970417,33.523583,제주도 북동부·우도,비양도 북측해상,암,22.0~30.0,제주특별자치도 제주시,0,DT_0004,제주


In [6]:
matrix = data.pivot_table(index='userId', columns='pointName', values='rating')
matrix.head(20)
print(matrix.columns)


Index(['.솔섬 맞은편', '1. 증서 부근', '2. 광서 부근', '3. 도토머리도 부근', '6.수월약수터 앞 갯바위',
       '가도 부근', '가도 북측 갯바위', '가동서 부근', '가두암 부근', '가란도',
       ...
       '흑도와 골서 부근', '흑서 부근', '흑여(낚시배)', '흑초 부근', '흔들바위 상단', '흥남해수욕장 몽돌밭',
       '흥어굴 인근', '흰여', '흰여 부근', '흰여 해상'],
      dtype='object', name='pointName', length=1305)


In [31]:
obsPostId_WEIGHT = 0.1

def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_pointName, matrix, n, similar_genre=True):
    fishingId = meta[meta['pointName'] == input_pointName]['fishingId'].iloc(0)[0]
    print("fishingId는--------------- ", fishingId)
    input_obsPostId = meta[meta['pointName'] == input_pointName]['obsPostId'].iloc(0)[0]
    result = []
    for title in matrix.columns:
        if title == input_pointName:
            continue

        # rating comparison
        cor = pearsonR(matrix[input_pointName], matrix[title])
        
        # genre comparison
        if similar_genre and len(input_obsPostId) > 0:
            temp_obsPostId = meta[meta['pointName'] == title]['obsPostId'].iloc(0)[0]
            same_count = np.sum(np.isin(input_obsPostId, temp_obsPostId))
            cor += (obsPostId_WEIGHT * same_count)

        if np.isnan(cor):
            continue
        else:       
            result.append((title, '{:.2f}'.format(cor), temp_obsPostId, fishingId))
        

            
    result.sort(key=lambda r: r[1], reverse=True)
    result[:n]
    return result[:n]
    

recommend_result = recommend('거북바위', matrix, 3, similar_genre=True)
# print(recommend_result)
pd.DataFrame(recommend_result, columns = ['pointName', 'Correlation', 'obsPostId', 'fishingId'])



fishingId는---------------  816


<ipython-input-31-c130f3d3d7df>:6: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


,pointName,Correlation,obsPostId,fishingId
0,막두,0.30,여수,816
1,장산넘어 어장줄 자리,0.30,진도,816
2,작은 마을 동측 안통,0.22,태안,816


In [9]:

# lst는 실제로 본인이 평점 좋게 준 낚시터들 상위 3개를 가지고 옴

lst = ['달천도 부근', '낭도산 동측', '거북바위']
res = []
for i in range(len(lst)):
    recommend_result = recommend(lst[i], matrix, 3, similar_genre=True)
    res.append(pd.DataFrame(recommend_result, columns = ['pointName', 'Correlation', 'obsPostId']))
    
abc = pd.concat(res)
abc.sort_values(by=['Correlation'], axis=0, ascending=False)
     

input_obsPostId는 순천만


<ipython-input-8-c0c9afc97473>:6: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


[('화단도', '0.21', '진도'), ('우세도 부근', '0.20', '목포'), ('신흥 부근', '0.18', '고흥발포')]
input_obsPostId는 여수
[('복용리해안 철탑 밑', '0.37', '진도'), ('오동여, 검등여 사이 부근 해상', '0.34', '신안가거초'), ('금오도 학동마을 이장바위', '0.33', '여수')]
input_obsPostId는 진도
[('막두', '0.30', '여수'), ('장산넘어 어장줄 자리', '0.30', '진도'), ('작은 마을 동측 안통', '0.22', '태안')]


,pointName,Correlation,obsPostId
0,복용리해안 철탑 밑,0.37,진도
1,"오동여, 검등여 사이 부근 해상",0.34,신안가거초
2,금오도 학동마을 이장바위,0.33,여수
0,막두,0.30,여수
1,장산넘어 어장줄 자리,0.30,진도
2,작은 마을 동측 안통,0.22,태안
0,화단도,0.21,진도
1,우세도 부근,0.20,목포
2,신흥 부근,0.18,고흥발포
